In [1]:
import pandas as pd
import psycopg2
import json
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import markers

In [2]:
sns.set_style('whitegrid')
all_shapes = list(markers.MarkerStyle.markers.keys())
f = open('/Users/aliglara/Documents/MyGit/apis/credential_keys.json', "r")
keys = json.load(f)

# Connection to dataset on Heroku

In [3]:
def connect_database(server, project, credentials):
    conn = psycopg2.connect(
        host=credentials['databases'][server][project]['host'],
        user=credentials['databases'][server][project]['user'],
        password=credentials['databases'][server][project]['password'],
        database=credentials['databases'][server][project]['database'],
        port=credentials['databases'][server][project]['port']
    )
    return conn.cursor()

In [6]:

#database.close()
# %%
list_states = ['NY', 'CA', 'MA', 'WA', 'PA', 'OH', 'AZ', 'TX', 'OR', 'IL', 'MI', 'CO', 'GA', 'FL']

string_name = ",".join(list_states)

query = ("SELECT robi.*, sr.abbreviation "
         "FROM rent_occupied_by_income as robi "
         "JOIN state_region sr ON robi.id_state = sr.state "
         "WHERE sr.abbreviation IN "
         "('NY', 'CA', 'MA', 'WA', 'PA', 'OH', 'AZ', 'TX', 'OR', 'IL', 'MI', 'CO', 'GA', 'FL')")

In [8]:

cursor = connect_database("heroku", "c1_capstone", keys)
cursor.execute(query)

myresult = cursor.fetchall()

cursor.close()
column_names = [i[0] for i in cursor.description]
rent_cost_income_df = pd.DataFrame(myresult, columns=column_names)
#rent_cost_income_df = rent_cost_income_df.drop(['id', 'state', 'Name'], axis=1)

In [9]:
rent_cost_income_df

,id,year,name_state,less_than_r5000,r5000_to_r9999,r10000_to_r14999,r15000_to_r19999,r20000_to_r24999,r25000_to_r34999,r35000_to_r49999,r50000_to_r74999,r75000_to_r99999,r100000_to_r149999,r150000_or_more,id_state,abbreviation
0,9,2015,New York,203410,253643,275686,243069,214353,369940,435593,528287,310475,304390,255946,36,NY
1,12,2015,Ohio,117215,139631,154696,143831,129371,228646,235353,245058,108598,65339,25824,39,OH
2,14,2015,Oregon,32209,41893,48218,47915,46979,82908,97386,103321,49847,36362,17276,41,OR
3,15,2015,Pennsylvania,93232,125261,143444,133714,117588,206567,230961,247278,121478,86449,44309,42,PA
4,20,2015,Texas,194963,218038,258383,268064,279914,503234,593581,645289,332113,245359,129648,48,TX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,285,2020,Texas,193418,184862,223591,224888,235369,464296,585999,713123,388891,343101,179724,48,TX
86,288,2020,Ohio,99082,116293,132465,123037,121083,212852,250741,268085,134932,93271,37213,39,OH
87,289,2020,Colorado,30144,28160,40104,38106,40976,77651,107303,147626,87087,81668,43253,8,CO
88,296,2020,Washington,40998,44458,58210,52070,56737,105126,154729,202831,128310,132307,91987,53,WA


In [ ]:

rent_cost_income_df.head()
fig, ax = plt.subplots(figsize=(10, 8))

_ = sns.barplot(x='Abbreviation', y='Total Renter occupied - by income',
                data=rent_cost_income_df[rent_cost_income_df['Year'] == 2015],
                ax=ax)
ax.set_xlabel('US State', fontsize=12)
ax.set_ylabel('Total Number of rental houses', fontsize=12)
plt.show()

#pd.pivot_table(rent_cost_income_df.loc['2015'], index=['Abbreviation'])